In [0]:
!wget https://github.com/v3rm1/ml_bach_fugue/archive/midi_to_txt.zip
!unzip midi_to_txt.zip

--2020-01-07 18:55:15--  https://github.com/v3rm1/ml_bach_fugue/archive/midi_to_txt.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/v3rm1/ml_bach_fugue/zip/midi_to_txt [following]
--2020-01-07 18:55:16--  https://codeload.github.com/v3rm1/ml_bach_fugue/zip/midi_to_txt
Resolving codeload.github.com (codeload.github.com)... 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘midi_to_txt.zip.1’

midi_to_txt.zip.1       [ <=>                ]  16.35K  89.5KB/s    in 0.2s    

2020-01-07 18:55:17 (89.5 KB/s) - ‘midi_to_txt.zip.1’ saved [16744]

Archive:  midi_to_txt.zip
2e335f8c730465a975d52cf4ba21a1bfb963c12d
   creating: ml_bach_fugue-midi_to_txt/
  inflating: ml_bach_fugue-midi_t

In [0]:
!mv ml_bach_fugue-midi_to_txt/preprocessing/converters/ /content/
!rm -rf ml_bach_fugue-midi_to_txt/

mv: cannot move 'ml_bach_fugue-midi_to_txt/preprocessing/converters/' to '/content/converters': Directory not empty


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from bs4 import BeautifulSoup
import requests
import os
import glob
import zipfile

url = 'http://www.stringquartets.org/composers.htm'
outpath = '/content/gdrive/My Drive/raw_data/'
os.makedirs(outpath, exist_ok=True)
r = requests.get(url)
soup = BeautifulSoup(r.text)
for link in soup.find_all('a', href=True):
  zipurl = link['href']
  if(zipurl.endswith('.zip')):
    outfname = outpath + zipurl.split('/')[-1]
    r = requests.get('http://www.stringquartets.org/' + zipurl, stream=True)
    if( r.status_code == requests.codes.ok ) :
        fsize = int(r.headers['content-length'])
        print('Downloading {}'.format(outfname))
        with open(outfname, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024): # chuck size can be larger
                if chunk: # ignore keep-alive requests
                    fd.write(chunk)
            fd.close()

for files in glob.iglob('/content/gdrive/My Drive/raw_data/*.zip'): # Extract files
  f = zipfile.ZipFile(files)
  f.extractall('/content/gdrive/My Drive/raw_data')
  f.close()
  os.remove(files)

for files in glob.iglob('/content/gdrive/My Drive/raw_data/*.kpp'): # Remove .kpp files
  os.remove(files)

for files in glob.iglob('/content/gdrive/My Drive/raw_data/*.squ'): # Rename .squ files to midi
  name = files.split('/')[-1].split('.')[0]
  os.rename(files, '/content/gdrive/My Drive/raw_data/'+name+'.mid')

In [0]:
!pip install music21 mido

In [0]:
""" Function taken from 
http://nickkellyresearch.com/python-script-transpose-midi-files-c-minor/
"""

def transpose_key(filename):
    """ converting everything into the key of C major or A minor
    """
    # major conversions
    majors = dict([('A-', 4),('G#', 4),('A', 3),('A#', 2),('B-', 2),('B', 1),('C', 0),('C#', -1),('D-', -1),('D', -2),
            ('D#', -3),('E-', -3),('E', -4),('F', -5),('F#', 6),('G-', 6),('G', 5)])
    minors = dict([('G#', 1), ('A-', 1),('A', 0),('A#', -1),('B-', -1),('B', -2),('C', -3),('C#', -4),('D-', -4),
            ('D', -5),('D#', 6),('E-', 6),('E', 5),('F', 4),('F#', 3),('G-', 3),('G', 2)])

    score = music21.converter.parse(filename)
    key = score.analyze('key')
    #    print key.tonic.name, key.mode
    if key.mode == "major":
        halfSteps = majors[key.tonic.name]
    elif key.mode == "minor":
        halfSteps = minors[key.tonic.name]

    newscore = score.transpose(halfSteps)
    newscore.write('midi', '/tmp/temp.mid')

In [0]:
import converters.convert as cvt
from mido import MidiFile, MidiTrack, Message as MidiMessage
import music21
import tqdm

print('Total files found: ', len(glob.glob('/content/gdrive/My Drive/raw_data/*.mid')))

for file in tqdm.tqdm(glob.iglob('/content/gdrive/My Drive/raw_data/*.mid')):
    try:
      transpose_key(file)
    except:
      print('Error occured while reading file\n Deleting file')
      os.remove(file)
      continue
    
    mid = MidiFile('/tmp/temp.mid')
    tpb = mid.ticks_per_beat
    metamessages, insmessages = cvt.split_tracks(mid)  # Make sure that only 4 instruments are passed forward
    streams = cvt.notes_to_pitchstream(insmessages, tpb)
    cvt.pitch_stream_to_text(streams, file_name='/content/gdrive/My Drive/corpus.txt')
    os.remove(file)




0it [00:00, ?it/s]

Total files found:  353



1it [00:28, 28.17s/it]
2it [00:38, 22.94s/it]
3it [01:02, 23.22s/it]
4it [01:22, 22.09s/it]
5it [01:40, 20.88s/it]
6it [01:48, 16.95s/it]
7it [02:02, 16.33s/it]
8it [02:27, 18.77s/it]
9it [03:03, 23.82s/it]

Error occured while reading file
 Deleting file
Error occured while reading file
 Deleting file



12it [03:22, 18.63s/it]
13it [03:37, 17.40s/it]
14it [04:01, 19.64s/it]
15it [04:08, 15.61s/it]
16it [04:38, 19.93s/it]
17it [04:59, 20.27s/it]
18it [05:05, 15.97s/it]
19it [05:29, 18.35s/it]
20it [06:10, 25.14s/it]
21it [06:19, 20.52s/it]
22it [06:25, 15.97s/it]

...


348it [1:56:08, 17.46s/it]

Found tracks having more than 4 instruments!
 Removing instruments
Found tracks having more than 4 instruments!
 Removing instruments



349it [1:56:18, 15.38s/it]
350it [1:56:58, 22.51s/it]
351it [1:57:13, 20.46s/it]
352it [1:57:21, 16.53s/it]

Found tracks having more than 4 instruments!
 Removing instruments
Found tracks having more than 4 instruments!
 Removing instruments



353it [1:57:30, 14.48s/it]


In [0]:
!rm -rf /content/gdrive/My Drive/raw_data

In [0]:
stream = cvt.text_to_pitch_stream(file_name='')
mid = cvt.pitchstream_to_midi(stream)
mid.save('txt.mid')